# NLOS

## HHL Implementation

In [1]:
import math
import warnings
import numpy as np
import qiskit
from scipy.linalg import expm
from qiskit.circuit.library import phase_estimation, StatePreparation, ExactReciprocalGate
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
from qiskit.result import Result
from qiskit.circuit import IfElseOp, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import Initialize , UnitaryGate , QFTGate, ExactReciprocalGate

In [ ]:
# todo: if A is not hermitian, do [[0, A], [A, 0]] to make it hermitian

def progress_callback(**kwargs):
    pass_name = kwargs['pass_'].name()
    count = kwargs['count']
    # The DAG circuit is an internal representation that can be inspected
    dag = kwargs['dag'] 
    print(f"Pass {count}: {pass_name}, Operations: {dag.count_ops()}")

class HHL:
    """class that implements HHL algorithm"""

    def __init__(self, A: np.typing.NDArray, b: np.typing.NDArray, state_dim: int = -1, tol=1e-05):
        """A: operator matrix, b: matrix, n: number of qubits"""
        assert (len(A.shape) == 2 and len(b.shape) == 1), "Inconsistent dimension of A or b"
        assert (A.shape[0] == b.shape[0]), "shape of A does not match b"
        
        if (A.shape[0] != A.shape[1] or not np.allclose(A, A.conj().T, atol=tol)):
            if (A.shape[0] != A.shape[1]):
                print("A is not a square matrix")
            else:
                print("A is not Hermitian")
            self.A = np.block([[np.zeros((A.shape[0], A.shape[0])), A],
                               [A.conj().T, np.zeros((A.shape[1], A.shape[1]))]])
            
            print("A is evaluated as [[0, A], [AT, 0]]", "-----------------------------------", sep="\n")
            self.b = np.block([b, np.zeros(A.shape[1])])
        self.A
        self.n_b = n_b + 1
        self.dim = (1 << (n_b + 1))
    
        assert (A.shape[0] == A.shape[1] and  and A.shape[0] == (1 << n_b)), "Inconsistent shape: A is not N * N or b is not N * 1"

        self.tol = tol

        self.A = A.copy()   # A is of N * N, b is of (N,)
        self.b = b.copy()
        self.n_b = n_b
        self.dim = (1 << n_b)
            
        
        if (not math.isclose(np.linalg.norm(self.b), 1, abs_tol=tol)):
            warnings.warn("b is not normalized, resolved during init")
            self.b = self.b.copy() / np.linalg.norm(self.b)
        
        
        # C for making C/λ < 1
        eigvals = np.linalg.eigvalsh(self.A)
        self.C = np.abs(eigvals).min() * 0.9
        # todo: bound eigvals and avoid direct computation

        self.kappa = np.abs(eigvals).max() / np.abs(eigvals).min() # characteristic
        self.t = np.pi / np.abs(eigvals).max() # calculate t

        if (state_dim > 0):
            self.state_dim = state_dim
        else:
            self.state_dim = int(np.log2(self.kappa)) + 10

        self.U = UnitaryGate(expm(1j * self.t * self.A)) # U = e^(iAt), then convert to unitarygate


    
    def _create_register(self):
        """call this to initially create registers, only once"""
        b_reg = QuantumRegister(self.n_b, 'sys')
        c_reg = QuantumRegister(self.state_dim, 'phase')
        aux_reg = QuantumRegister(1, 'aux')
        classic_aux_reg = ClassicalRegister(1, 'classic_aux')
        result_reg = ClassicalRegister(self.n_b, 'classic_sys')
        return b_reg, c_reg, aux_reg, classic_aux_reg, result_reg


    def _prepare_state(self, circ: QuantumCircuit, b_reg: QuantumRegister):
        """add gate to mount b on registers"""
        # assert math.isclose(np.linalg.norm(self.b), 1, abs_tol=self.tol) # check if b is normalized
        init_gate = StatePreparation(Statevector(self.b))
        circ.append(init_gate, b_reg) # add gate for initalizing b
        circ.barrier() # separation from rest of the process


    def _controlled_rotation(self, circ, c_reg, aux_reg):
        """add controlled rotation block to circuit"""
        rot_gate = ExactReciprocalGate(self.state_dim, self.C * self.t / (2 * np.pi))
        circ.append(rot_gate, [*reversed(c_reg), *aux_reg])
    

    def _qpe(self, circ, b_reg, c_reg, inverse=False):
        """add QPE/IQPE gate to the circuit"""
        qpe_circ = phase_estimation(self.state_dim, self.U)
        if (inverse):
            qpe_circ = qpe_circ.inverse()
        circ.append(qpe_circ.to_gate(), [*c_reg, *b_reg]) # order may change


    def _measure(self, circ: QuantumCircuit, aux_reg: QuantumRegister, b_reg: QuantumRegister, classic_aux_reg: ClassicalRegister, result_reg: ClassicalRegister):
        """measure the quantum qubits and store in classic bits"""
        circ.measure(aux_reg, classic_aux_reg)
        circ.measure(b_reg, result_reg)

    def transpile(self, simulator=AerSimulator):
        """function to transpile circuit. required to be executed before run"""
        b_reg, c_reg, aux_reg, classic_aux_reg, result_reg = self._create_register()


        circ = QuantumCircuit(aux_reg, c_reg, b_reg, result_reg, classic_aux_reg)
        self._prepare_state(circ, b_reg)
        self._qpe(circ, b_reg, c_reg)
        circ.barrier()
        self._controlled_rotation(circ, c_reg, aux_reg)
        circ.barrier()
        self._qpe(circ, b_reg, c_reg, inverse=True)
        circ.barrier()
        self._measure(circ, aux_reg,  b_reg, classic_aux_reg, result_reg)

        self.backend = simulator()
        return transpile(circ, backend=self.backend, callback=progress_callback), circ
    
    def run(self, circ, shots=1):
        try:
            return self.backend.run(circ, shots=shots).result()
        except AttributeError:
            raise AttributeError("Circuit is not Transpiled before running")
    
    def transpile_and_run(self, shots=1):
        transpiled_circ = self.transpile()
        return self.run(transpiled_circ, shots=shots).result()
    
    def analyze(self, result: Result) -> tuple[float, np.ndarray]:
        count = result.get_counts()

        if (isinstance(count, dict)):
            # print(count)
            success = {res: cnt for res, cnt in count.items() if res.split()[0] == '1'}
            # failure = {res: cnt for res, cnt in count.items() if res.split()[0] == '0'}
            tot = sum(success.values())
            success_rate = tot / sum(count.values())
            
            x = np.zeros((1 << self.n_b), dtype=float)
            for res, cnt in success.items():
                ind = int(res.split()[1], 2)
                x[ind] = cnt / tot
            return success_rate, x
        else:
            print(count)
            raise TypeError("result.get_counts() is not type dict[str, int]")

In [3]:
A = np.array([
    [4, 1, -2, 3, 0, 5, -1, 2, 0, 4, -3, 1, 2, 0, -1, 6],
    [1, 7, 4, -1, 3, 0, 2, -2, 1, 3, 0, -3, 4, 2, 1, -2],
    [-2, 4, 5, 2, -3, 1, 0, 3, 2, 1, -1, 4, 0, -2, 3, 1],
    [3, -1, 2, 6, 1, -3, 4, 0, -2, 3, 2, 1, -1, 4, 0, 2],
    [0, 3, -3, 1, 8, 2, 1, -1, 4, 0, 3, -2, 5, 1, -3, 4],
    [5, 0, 1, -3, 2, 7, -2, 3, 0, 4, 1, -1, 3, -2, 4, 1],
    [-1, 2, 0, 4, 1, -2, 9, 1, 3, -1, 2, 0, 4, 3, -2, 1],
    [2, -2, 3, 0, -1, 3, 1, 6, 2, 1, -3, 4, 0, 1, 3, -1],
    [0, 1, 2, -2, 4, 0, 3, 2, 5, 3, -1, 1, 2, 0, 4, -3],
    [4, 3, 1, 3, 0, 4, -1, 1, 3, 10, 2, -3, 4, -1, 2, 1],
    [-3, 0, -1, 2, 3, 1, 2, -3, -1, 2, 6, 4, 0, 3, -2, 4],
    [1, -3, 4, 1, -2, -1, 0, 4, 1, -3, 4, 7, 1, -1, 3, 0],
    [2, 4, 0, -1, 5, 3, 4, 0, 2, 4, 0, 1, 9, 2, -3, 1],
    [0, 2, -2, 4, 1, -2, 3, 1, 0, -1, 3, -1, 2, 8, 1, 4],
    [-1, 1, 3, 0, -3, 4, -2, 3, 4, 2, -2, 3, -3, 1, 7, -1],
    [6, -2, 1, 2, 4, 1, 1, -1, -3, 1, 4, 0, 1, 4, -1, 11],
], dtype=float)
b = np.array([3, 1, -2, 4, 0, 5, -1, 2, 3, -3, 1, 0, 4, -2, 6, 1], dtype=float)

alg = HHL(A, b, 4)
circ, c = alg.transpile()

C:\Users\Dho\AppData\Local\Temp\ipykernel_43968\424633071.py:35: UserWarning: b is not normalized, resolved during init
  warnings.warn("b is not normalized, resolved during init")


Pass 0: ContainsInstruction, Operations: {'state_preparation': 1, 'barrier': 4, 'QPE': 1, 'ExactReciprocal': 1, 'QPE_dg': 1, 'measure': 5}
Pass 1: UnitarySynthesis, Operations: {'state_preparation': 1, 'barrier': 4, 'QPE': 1, 'ExactReciprocal': 1, 'QPE_dg': 1, 'measure': 5}
Pass 2: HighLevelSynthesis, Operations: {'multiplexer_dg': 2, 'unitary': 12, 'cx': 70026, 'barrier': 4, 'h': 704, 'circuit-51': 1, 'rz': 5060, 'circuit-65': 1, 'circuit-71': 1, 'circuit-61': 1, 'circuit-125': 1, 'circuit-139': 1, 'circuit-145': 1, 'circuit-135': 1, 'circuit-161': 1, 'circuit-175': 1, 'circuit-181': 1, 'circuit-171': 1, 'circuit-91': 1, 'circuit-105': 1, 'circuit-111': 1, 'circuit-101': 1, 'circuit-207': 1, 'circuit-221': 1, 'circuit-227': 1, 'circuit-217': 1, 'circuit-281': 1, 'circuit-295': 1, 'circuit-301': 1, 'circuit-291': 1, 'circuit-317': 1, 'circuit-331': 1, 'circuit-337': 1, 'circuit-327': 1, 'circuit-247': 1, 'circuit-261': 1, 'circuit-267': 1, 'circuit-257': 1, 'circuit-363': 1, 'circuit-3

In [4]:
for shots in [10**x for x in range(3, 6)]:
    print(f"shots: {shots}")
    result = alg.run(circ, shots=shots)

    p, x = alg.analyze(result)
    print(p, x, sep='\n')

# classic
x_classic = np.linalg.solve(A, b)
print(x_classic ** 2 / np.linalg.norm(x_classic) ** 2)

shots: 1000
0.05
[0.02 0.14 0.02 0.16 0.   0.02 0.   0.24 0.02 0.04 0.06 0.   0.12 0.
 0.16 0.  ]
shots: 10000
0.0415
[0.06506024 0.13253012 0.01927711 0.20963855 0.01445783 0.04096386
 0.         0.08433735 0.03855422 0.02891566 0.06506024 0.00481928
 0.17590361 0.         0.12048193 0.        ]
shots: 100000
0.04153
[0.0659764  0.1343607  0.01757765 0.15892126 0.01252107 0.03876716
 0.00192632 0.11774621 0.0235974  0.02696846 0.06212377 0.00842764
 0.17866602 0.00337106 0.14880809 0.00024079]
[0.06612376 0.14503759 0.02576387 0.13811406 0.01961338 0.04738953
 0.00046719 0.13941361 0.02951867 0.01657731 0.07970613 0.00495379
 0.15609111 0.00063003 0.12950446 0.00109553]
